# Football Actor-Critic

Partiendo de la base de un trabajo final con el algoritmo ***Asynchronous Advantage Actor Critic (A3C)*** para la [materia *Procesos Markovianos para el Aprendizaje Automático*](http://www.ic.fcen.uba.ar/actividades-academicas/formacion/cursos/procesos-markovianos-para-aprendizaje-automatico) dictada en Exactas, decidí adaptarlo para el entorno *football* a modo de comprobar qué tan bien generaliza.

A3C **reemplaza el uso de memoria** de experiencias de modelos como DQN, **con varios procesos asincrónicos** devolviendo gradientes de sus respectivas redes locales, copias del modelo global.

De esta forma se intenta acercar al modelo de Aprendizaje Supervisado, no muestreando sobre batchs de datos históricos, sino sobre secuencias de datos no tan correlacionadas entre sí.

Modelo anterior para entrada de pixeles:
![actor-critic-model-lstm](https://raw.githubusercontent.com/LecJackS/TP-Final-Procesos-Markovianos-para-el-Aprendizaje-Automatico-2019-1C-w.o.heavy-history-/master/img/actor-critic-model-lstm.png)

La diferencia ahora es que la entrada serán features numéricas directamente, por lo que pueden descartarse las capas convolucionales y pasar directamente como features a las capas lineales, o para problemas/entornos con mayores relaciones temporales, una capa LSTM.

Para el entorno de *football* se agregaron dos capas lineales de 64 unidades a modo de relacionar features diferentes entre sí (principalmente las de posiciones cardinales (x,y)), conectada a una capa lineal en lugar de la LSTM.

![actor-critic-football-model.png](./img/actor-critic-football-model.png)

El algoritmo elegido es Asynchronous Advantage Actor Critic (A3C), ya que brinda la posibilidad de aprovechar más eficientemente el uso del CPU, sin usar GPU.

1. Los multiprocesos (**LOCALES**) obtienen una **copia** de los parámetros del modelo neuronal **GLOBAL** al iniciar la partida.

2. Operan con esa copia hasta terminar la partida o definición de episodio.

3. Calculan *Loss* similarmente a Q-Learning/SarsaMax.

4. Actualizan los parámetros **GLOBALES** directamente con optimizador Adam.

5. Repite desde 1., con varios procesos funcionando en paralelo.

Esto sucede todo entre CPU y memoria, por lo que las actualizaciones y cálculos de gradientes se realizan a gran velocidad.

![a3c](./img/a3c.png)

# Índice de archivos

En la notebook se encuentra la definición del agente y sus parámetros globales, pero fue necesario dividir parte del código en archivos individuales, por requerimientos de las librerías de multiprocesos de pytorch.

1. ***local_train.py:*** Definición de un **proceso asincrónico particular**. Será usado por *train()* definida abajo en la notebook, para generar los actores.


2. ***local_test.py:*** Similar a local_train, solo que **hace uso pasivo de los parámetros** de un nuevo modelo asincrónico (no actualiza/aprende ni calcula gradientes).


3. ***env.py:*** Contiene *create_train_env()* que se encarga de **generar un entorno para football** (o cualquier otro env)


4. ***model.py:*** **Modelo neuronal** con una capa lineal que recibe las features del entorno, conectada a otras dos capas individuales que reprensentan el Actor y el Crítico.
 
  El ***Actor*** tendrá **una salida por cada acción** posible, que representa el **puntaje** de la misma, que se usará para calcular ***policy softmax***.
  
  El ***Crítico*** tendrá **una sola salida**: El valor de la **value function** para ese estado


5. ***optimizer.py:*** Se definen los mismos optimizadores Adam y RMSProp de Pytorch, pero con **parámetros compartidos entre procesos asincrónicos y parámetros globales**.


Para más detalles del funcionamiento del algoritmo A3C, dejo la notebook del trabajo realizado anteriormente mencionado, con más definiciones teóricas e intuición detrás del mismo:

[Very quick roadmap to Asynchronous Advantage Actor Critic.ipynb](https://github.com/LecJackS/TP-Final-Procesos-Markovianos-para-el-Aprendizaje-Automatico-2019-1C-w.o.heavy-history-/blob/master/Very%20quick%20roadmap%20to%20Asynchronous%20Advantage%20Actor%20Critic.ipynb)

# Sobre gfootball

Todos los valores del estado (para *simple115*) son valores entre -1 y 1, por lo que se decidió dejarlos de esa forma.

Para una entrada de píxeles los valores de cada uno están entre 0 y 255, que suelen normalizarse según la media y varianza de la imagen completa (o un batch), por lo que se esperan ciertas diferencias en cuanto al aprendizaje inicial del agente.

Los rewards son también de -1 y 1.

En *env.py* se definió un *wrapper* sobre el cual se puede controlar con más detalle a los rewards, agregando también descuentos por cada timestep.

Los resultados de varias pruebas indicaron que penalizar por tiempo retraza (al menos al comienzo) el aprendizaje del agente.

La causa de ésto posiblemente esté relacionada con lo que menciona Vlad Mnih en el siguiente video:

***Reinforcement Learning 9: A Brief Tour of Deep RL Agents*** @1:31:50 https://youtu.be/-mhBD8Frkc4?t=5515

Donde explica que **rewards simples** del tipo "buena o mala" acción, **facilitan el aprendizaje** simplificando el problema a **obtener una mayor cantidad de acciones positivas**.

Agregando penalización por tiempo los rewards serán valores muy variables, y por más que estén en un rango fijo (ej -1, 1) el agente tiene dificultad para aprender de ellos.

# Experimentación

A pesar de verse mejoras en el tiempo con rewards simples sin penalidad, cada cierto intervalo se notó un decaimiento en performance, hasta alcanzar valores muy bajos sin recuperarse.

A continuación, plots de:

1. **Reward acumulado** por cada episodio
2. **Reward promedio** de los pasados **100 episodios**
3. **Mediana** del los últimos **100 episodios**
![big-lr.png](./img/big-lr.png)

Notar los picos en plots de reward promedio, poco después de los 500 episodios, 1300 y 1600.

Luego de algunas pruebas con resultados similares, era indicio de un learning rate muy alto, resultando en divergencia por pasos demasiado grandes.

Se decidió por reiniciar el aprendizaje desde cero, pero a los 500 episodios, **se redujo el learning rate** en un órden de magnitud (de 1e-4 a 1e-5).

![smaller-lr.png](./img/smaller-lr.png)

Este cambio produjo resultados muy positivos en la estabilidad del algoritmo en el tiempo, permitiendo mantenerse entre el 60-80% de juegos ganados, con tendencia a seguir aumentando en el tiempo.



# TO-DO: 

Otros modelos más complejos (con LSTM) fueron probados **con todos los mapas** de manera aleatoria en cada episodio, observando muy pocas mejoras.

Experimentos limitados al mapa simple permitieron ver que un modelo con una capa de LSTM **no** obtiene los resultados que se llegan a obtener con una simple feedforward layer.

Ésto indica que falta regular hiperparámetros o simplemente necesita mucho más tiempo para comenzar a mostrar resultados.

Se intentó explorar ambas posibilidades durante estas semanas, corriendo tres experimentos a la vez en tres computadoras, pero no fue suficiente.

Faltaría más análisis en el problema para encontrar posibles causas y ser eficiente con el tiempo invertido.

# Código

Las siguientes 3 celdas de código permiten entrenar un agente para un modelo lineal desde cero.

Para elegir otro modelo basta modificar AC_NN_MODEL en *local_train.py* (y en la celda de Interfaz abajo) con los modelos neuronales disponibles en *model.py*

Las últimas 2 celdas ejecutan un agente en modo *Test* ya entrenado.

# Global parameters: Agent definition

Se define el modelo de parámetros globales, y los procesos asincrónicos que actuarán como *actors* en el proceso de aprendizaje, devolviendo gradientes de experiencias, en forma de updates paralelos asincrónicos.

In [1]:
def train(opt):
    torch.manual_seed(42)
    # Prepare log directory
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
    os.makedirs(opt.log_path)
    # Prepare saved models directory
    if not os.path.isdir(opt.saved_path):
        os.makedirs(opt.saved_path)
    # Prepare multiprocessing
    mp = _mp.get_context("spawn")
    # Create new training environment just to get number
    # of inputs and outputs to neural network
    _, num_states, num_actions = create_train_env(opt.layout, opt.num_processes_to_render)
    # Create Neural Network model
    global_model = AC_NN_MODEL(num_states, num_actions)
    if opt.use_gpu:
        global_model.cuda()
    # Share memory with processes for optimization later on
    global_model.share_memory()
    # Load trained agent weights
    if opt.load_previous_weights:
        file_ = "{}/gfootball_{}".format(opt.saved_path, opt.layout)
        if os.path.isfile(file_):
            print("Loading previous weights for %s..." %opt.layout, end=" ")
            # global_model.load_state_dict(torch.load(file_))
            pretrained_dict = torch.load(file_)
            global_model_dict = global_model.state_dict()
            # 1. filter out unnecessary keys (if trained with different model)
            pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in global_model_dict}
            
            # 2. overwrite entries in the existing state dict
            global_model_dict.update(pretrained_dict) 
            # 3. load the new state dict (only existing keys)
            global_model.load_state_dict(pretrained_dict, strict=False)
            print("Done.")
        else:
            print("Can't load any previous weights for %s! Starting from scratch..." %opt.layout)
    # Define optimizer with shared weights. See 'optimizer.py'
    optimizer = GlobalRMSprop(global_model.parameters(), lr=opt.lr)
    # Create async processes
    processes = []
    for index in range(opt.num_processes):
        # Multiprocessing async agents
        if index == 0:
            # Save weights to file only with this process
            process = mp.Process(target=local_train, args=(index, opt, global_model, optimizer, True))
            
        else:
            process = mp.Process(target=local_train, args=(index, opt, global_model, optimizer))
        process.start()
        processes.append(process)
        
    # Local test simulation (creates another model = more memory used)
    #process = mp.Process(target=local_test, args=(opt.num_processes, opt, global_model))
    #process.start()
    #processes.append(process)
    for process in processes:
        process.join()

# Interface

In [2]:
import os
# To NOT use OpenMP threads within numpy processes
os.environ['OMP_NUM_THREADS'] = '1'
import argparse
import torch
from model import SimpleActorCriticLineal
AC_NN_MODEL = SimpleActorCriticLineal
# For the async policies updates
import torch.multiprocessing as _mp
# _multiprocessing needs functions to be imported from their own file
# so it does not work if functions are defined on this jupyter notebook
from local_train import local_train #
from env import create_train_env
from optimizer import GlobalAdam, GlobalRMSprop
import shutil

# Train Agent

Basta correr la siguiente celda (luego de las anteriores) para simular una cantidad ***num_processes*** de procesos asincrónicos, pudiendo visualizarlas con el control  ***num_processes_to_render***.

En ./tensorboard se guardará una carpeta con estadísticas del agente.

Para visualizarla:

    tensorboard --logdir=some_id:./tensorboard/
    # or
    tensorboard --logdir=some_id:/home/USER/ECI2019-Aprendizaje-Profundo-por-Refuerzo/tensorboard/

In [7]:
from argparse import Namespace
args = Namespace(beta =0.01,
                 gamma=0.99,
                 tau=1.0,
                 lr=1e-4,
                 layout='desde_cero',
                 load_previous_weights=True,
                 num_processes=8,                # numero de procesos asincronicos
                 num_processes_to_render=1,     # numero de procesos a visualizar
                 use_gpu=False,
                 max_actions=200,
                 num_global_steps=5e9,
                 num_local_steps=50,     # async updates every
                 save_interval=10,
                 saved_path='trained_models',
                 log_path='tensorboard')
print("Los detalles del aprendizaje se muestran en la TERMINAL (not here)\n")
train(args)

Los detalles del aprendizaje se muestran en la TERMINAL (not here)

Getting number of NN inputs/outputs for desde_cero
Can't load any previous weights for desde_cero! Starting from scratch...


KeyboardInterrupt: 

# Test Agent

A continuación un agente entrenado con un modelo lineal que acierta la mayoría de las veces en el mapa de entrenamiento:

In [4]:
def test(opt):
    torch.manual_seed(42)
    # Prepare saved models directory
    if not os.path.isdir(opt.saved_path):
        os.makedirs(opt.saved_path)
    # Prepare multiprocessing
    mp = _mp.get_context("spawn")
    # Create new training environment just to get number
    # of inputs and outputs to neural network
    _, num_states, num_actions = create_train_env(opt.layout, 1)
    # Create Neural Network model
    global_model = AC_NN_MODEL(num_states, num_actions)
    # Share memory with processes for optimization later on
    global_model.share_memory()
    # Load trained agent weights
    file_ = "{}/gfootball-lineal-75".format(opt.saved_path)
    if os.path.isfile(file_):
        print("Loading previous weights for %s..." %opt.layout, end=" ")
        # global_model.load_state_dict(torch.load(file_))
        pretrained_dict = torch.load(file_)
        global_model_dict = global_model.state_dict()
        # 1. filter out unnecessary keys (if trained with different model)
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in global_model_dict}

        # 2. overwrite entries in the existing state dict
        global_model_dict.update(pretrained_dict) 
        # 3. load the new state dict (only existing keys)
        global_model.load_state_dict(pretrained_dict, strict=False)
        print("Done.")
    else:
        print("Can't load any previous weights for %s! Starting from scratch..." %opt.layout)
 
    #Local test simulation
    process = mp.Process(target=local_test, args=(1, opt, global_model))
    process.start()

#### Para ejecutar varios agentes al mismo tiempo (en modo test), ejecutar la siguiente celda varias veces:

In [5]:
from local_test import local_test
from argparse import Namespace
args = Namespace(layout='football',
                 load_previous_weights=True,
                 max_actions=200,
                 num_global_steps=5e9,
                 saved_path='trained_models')
print("Los detalles de los episodios se muestran en la TERMINAL (not here)\n")

test(args)

Los detalles de los episodios se muestran en la TERMINAL (not here)

Getting number of NN inputs/outputs for football
Loading previous weights for football... Done.
